In [1]:
from deap import base, creator, tools
import random

In [2]:
# Problem constants
# Length of bit string to be optimized
ONE_MAX_LENGTH = 100

In [3]:
# Genetic Algorithm constants
POPULATION_SIZE = 200
# Probability for crossover
P_CROSSOVER = 0.9
# Probability for mutating an individual
P_MUTATION = 0.1
MAX_GENERATIONS = 50

In [4]:
# Set the random seed
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [5]:
toolbox = base.Toolbox()
# Create an operator defining the alias of `random.randint(0, 1)` as `zeroOrOne`
toolbox.register("zeroOrOne", random.randint, 0, 1)

# Populate a `list` with `initRepeat`
# randomList = tools.initRepeat(list, toolbox.zeroOrOne, 30)
# print(randomList)

In [6]:
# Define the fitness strategy by extending the `base.Fitness` class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [7]:
# Create the `Individual` class based on `list`
creator.create("Individual", list, fitness=creator.FitnessMax)

In [8]:
# Create the `individualCreator` operator to fill up an `Individual` instance
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, ONE_MAX_LENGTH)

In [9]:
# Create the population operator to generate a list of individuals
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

In [10]:
# Fitness Calculation
# Compute the number of `1`s in the individual
def oneMaxFitness(individual):
    # Return a Tuple
    return sum(individual)

toolbox.register("evaluate", oneMaxFitness)

In [11]:
# Genetic Operators
# Tournament selection with tournament size of `3`
toolbox.register("select", tools.selTournament, tournsize=3)

In [12]:
# Single-point crossover
toolbox.register("mate", tools.cxOnePoint)

In [13]:
# Flip-bit mutation
# indpb: Independent probability for each attribute to be flipped
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ONE_MAX_LENGTH)

In [15]:
# Python `map()` function
def addition(n):
    return n + n

numbers = (1, 2, 3, 4)
result = map(addition, numbers)
print(list(result))

[2, 4, 6, 8]


In [14]:
# Genetic Algorithm flow:
def main():
    
    # Create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)
    generationCounter = 0
    
    # Calculate fitness tuple for each individual in the population
    fitnessValues = list(map(toolbox.evaluate, population))
    for individual, fitnessValue in zip(population, fitnessValues):
        individual.fitness.values = fitnessValue
        
    # Extract fitness values from all individuals in population
    fitnessValues = [individual.fitness.values[0] for individual in population]